## Model Template
This is an example template where you can insert your model implementation.

**Note:** Customize this template based on your specific model implementation and requirements. Insert your model below.

In [22]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from imblearn.under_sampling import RandomUnderSampler
import pickle

# Load and preprocess data
covid = pd.read_csv('Covid Data_30percent_3.csv')

# Clean data
cols = ['PNEUMONIA', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR', 'HIPERTENSION',
  'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY', 'RENAL_CHRONIC', 'TOBACCO']
for col in cols:
    covid = covid[(covid[col] == 1)|(covid[col] == 2)]

# Create death column
covid['DEATH'] = [2 if row=='9999-99-99' else 1 for row in covid['DATE_DIED']]
covid.drop(columns=['INTUBED', 'ICU', 'DATE_DIED'], inplace=True)
covid['PREGNANT'] = covid['PREGNANT'].replace({97:2, 98:2})

# Prepare features and target
y = covid['DEATH']
X = covid.drop('DEATH', axis=1)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Apply undersampling and split data
undersampler = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = undersampler.fit_resample(X, y)
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Train model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Get predictions
y_pred = model.predict(X_test)

# Calculate metrics
metrics = {
    'accuracy': accuracy_score(y_test, y_pred),
    'precision': precision_score(y_test, y_pred, average="weighted"),
    'recall': recall_score(y_test, y_pred, average="weighted"),
    'f1score': f1_score(y_test, y_pred, average="weighted")
}

# Save model parameters for aggregation
model_params = {
    'predictions': y_pred.tolist(),
    'coefficients': model.coef_[0].tolist(),
    'intercept': model.intercept_[0],
    'feature_names': X.columns.tolist() if hasattr(X, 'columns') else None,
    'scaler_params': {
  'scale_': scaler.scale_.tolist(),
  'mean_': scaler.mean_.tolist()
    }
}

# Save model parameters
with open('model_params.pkl', 'wb') as f:
    pickle.dump(model_params, f)

#save the model metrics in to a text file
with open('metrics.txt', 'w') as f:
  for key, value in metrics.items():
    f.write(f'{key}: {value}\n')

print("Metrics saved:", metrics)



Metrics saved: {'accuracy': 0.9041111111111111, 'precision': 0.904503609281313, 'recall': 0.9041111111111111, 'f1score': 0.904085231192201}


## Files Required for Backend Processing

The following files need to be generated and saved before sending to the backend:

 `metrics.txt` - Contains model performance metrics: (for both model & data owner)
  - Accuracy
  - Precision
  - Recall
  - F1 Score

 `model_params_encrypted` - Encrypted version of model parameters using Fernet encryption (only for data owner)

 `model_hash` - Contains IPFS hash of the model (only for model owner)

These files needs to be processed and combined into a JSON object containing: (For model Owners)
- model_ipfs_hash
- metrics

These files needs to be processed and combined into a JSON object containing: (For data Owners)
- metrics
- param_ipfs_hash
- param_key (base64 encoded encryption key)

This JSON data is then sent to the backend endpoint at http://127.0.0.1:5000/update-data

Below shown is an example of file generation and backend processing

In [23]:
#Read the saved model metrics
%pip install requests
import json
import requests


with open('metrics.txt', 'r') as file:
  metrics = file.read().strip()

metrics_dict = {}
for line in metrics.splitlines():
  if ': ' in line:
    key, value = line.split(': ', 1)
    metrics_dict[key] = int(float(value) * 100)

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Model Owner Section

**Note:** This section is specifically for model owners. If you are a data owner, please ignore or remove the cell below.

This section handles:
- Model IPFS upload
- Backend data preparation

In [24]:
# # #Upload the model on to IPFS
# %pip install python-dotenv

# import sys
# sys.path.append('../..')

# from ipfs_configs import upload_file_to_ipfs

# model_file_path = 'COVID-19-mortality-prediction-model.ipynb' # update your model file name here

# model_ipfs_hash = upload_file_to_ipfs(model_file_path)

# with open('model_hash', 'r') as file:
#   ipfs_hash = file.read().strip()

# # Create final JSON object
# data = {
#   'model_ipfs_hash': model_ipfs_hash,
#   'metrics': metrics_dict,
# }


# # Convert to JSON string
# json_data = json.dumps(data, indent=2)
# print("JSON data:", data)

# response = requests.post('http://127.0.0.1:5000/update-data', json=data)

## Data Owner Section

**Note:** This section is specifically for data owners. (Model Owners please keep section as it is, but comment it out when you run this template)

This section handles:
- Model metrics processing
- Parameter encryption
- IPFS upload
- Backend data preparation

In [25]:
#Encrypting Model Parameters & uploading it to IPFS

%pip install cryptography

import sys
sys.path.append('..')

from ipfs_configs import upload_file_to_ipfs
from cryptography.fernet import Fernet

key = Fernet.generate_key()
cipher_suite = Fernet(key)

with open('model_params.pkl', 'rb') as file:
  original_data = file.read()

encrypted_data = cipher_suite.encrypt(original_data)

with open('model_params_encrypted.pkl', 'wb') as file:
  file.write(encrypted_data)

params_file_path = 'model_params_encrypted.pkl'

param_ipfs_hash = upload_file_to_ipfs(params_file_path)

import base64

#Create final JSON object
data = {
  'metrics': metrics_dict,
  'param_ipfs_hash': param_ipfs_hash,
  'param_key': base64.b64encode(key).decode('utf-8')
}


# Convert to JSON string
json_data = json.dumps(data, indent=2)
print("JSON data:", data)

response = requests.post('http://127.0.0.1:5000/update-data', json=data)


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
JSON data: {'metrics': {'accuracy': 90, 'precision': 90, 'recall': 90, 'f1score': 90}, 'param_ipfs_hash': 'QmRXttzxiFZFCwcBAb4HzCdytPPDiJzBvvocats3fFofLF', 'param_key': 'OVdYYVJ3Zmh3YVoxcXc1THVlRXRhWmRvSWZnUWF1MXJPcnJ0c0taVWk2VT0='}
